# Projeto 2 - Ciência dos Dados

Nome: Cicero Tiago Carneiro Valentim

Nome: Luiz Felipe Lazzaron

Nome: Marcos Vinícius da Silva

# Introdução

No presente trabalho, visa-se selecionar tweets relevantes para a classificação de postagens que possuem sinais de depressão por parte de seus autores. Nesse contexto, os integrantes dividiram os tweets em três categorias:

1. Irrelavantes ou Neutros. Tweets que não indicam quaisquer indício de depressão;
2. Relevantes. Tweets que possuem relevância na análise de indicação de depressão;
3. Possuem interesse no assunto. Tweets que não demonstram doença mental, mas indicam um interresse no tema e na prevenção de suicídio.

Desse modo, com tal classificação, pode-se desenvolver um marketing específico para os públicos 2 e 3; para os relevantes, pode-se oferecer auxílio e suporte psicológico; para aqueles que possuem interesse no tema "depressão", pode-se direcionar materiais que ensinem a como lidar com pessoas com depressão.

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@MarcosV96118169***

In [2]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @MarcosV96118169

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [3]:
#Produto escolhido:
produto = 'Depressão'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [4]:
# Cria um objeto para a captura
# api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
#     new_msg = { 'User Name': msg.user.name,
#                 'Tweet Created At': msg.created_at,
#                 'Tweet Text': msg.full_text.lower(),
#                 'User Location': msg.user.location,
#                 'Phone Type': msg.source,
#                 'Favorite Count': msg.favorite_count,
#                 'Retweets':msg.retweet_count
#                 }
#     if i > n:
#         break

#Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

In [5]:
#Criando um Dataframe com a Base de Dados
# pd.set_option('display.max_colwidth', -1)
# pd.DataFrame(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#     writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [7]:
planilha_treinamento = pd.read_excel('depressao.xlsx', "Treinamento")
planilha_teste = pd.read_excel('depressao.xlsx', "Teste")
pd.set_option('display.max_colwidth', -1)

___
# Montando o Classificador Naive-Bayes


Primeiramente, iremos trabalhar com o dataframe treinamento, criando três banco de dados:
    
- **Grupo 1 - Irrelevantes/Neutros**: *group_1*
- **Grupo 2 - Relevantes**: *group_2*
- **Grupo 3 - Interesse no Assunto**: *group_3*


In [8]:
group_1 = planilha_treinamento[planilha_treinamento["Relevância"] == 1]

In [9]:
group_2 = planilha_treinamento[planilha_treinamento["Relevância"] == 2]

In [10]:
group_3 = planilha_treinamento[planilha_treinamento["Relevância"] == 3]

In [11]:
groups = [group_1,group_2,group_3]

# Limpeza e Tratamento dos Dados

#### Importando bibliotecas necessárias para análise:

In [13]:
#Bibliotecas para a Identificação de Emojis
!pip install emoji
from emoji import UNICODE_EMOJI
import emoji
import re 
import functools
import operator

### Função de Limpeza de um Tweet

In [14]:
# Recebe um texto normal e devolve uma lista de split
def space_cleaning(text):
    punctuation = '[!\-.:?;,''"@/]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    split_emoji = emoji.get_emoji_regexp().split(text_subbed)
    split_whitespace = [substr.split() for substr in split_emoji]
    split = functools.reduce(operator.concat,split_whitespace)
    return split

In [15]:
#Recebe um lista split e devolve uma lista split sem http ou https
def remove_http(lista_de_listas):
    lista = lista_de_listas
    for i in range(len(lista)):
        for j in range(len(lista[i])):
            if 'HTTP' in lista[i][j].capitalize():
                lista[i][j] = "http"
    
    for i in range(len(lista)):
        while lista[i].count("http") != 0:
            lista[i].remove("http")
    return lista

In [50]:
a = "Estou testando essa função https://facebook.com @marcelinhoparaiba"
b = space_cleaning(a)
c = remove_http(b)

AttributeError: 'str' object has no attribute 'remove'

### Tratando os Dados da Base de Treinamento

In [16]:
# groups_tweet = []
# for group in groups:
#     tweet_group = group['Tweet Text'] 
#     tweet_group = str(tweet_group)
#     tweet_group = space_cleaning(tweet_group)
#     groups_tweet.append(tweet_group)

In [17]:
def tratamento_do_grupo(grupo):
    total = []
    for tweet in grupo["Tweet Text"]:
        tweet_splited = space_cleaning(tweet)
       # tweet_splited = remove_http(tweet_splited)
        total.append(tweet_splited)
    return total

In [18]:
groups_tweet = [tratamento_do_grupo(groups[0]),tratamento_do_grupo(groups[1]),tratamento_do_grupo(groups[2])]

## Naive Bayes e Laplace Smoothing

In [19]:
#Número de Elementos De todos os grupos sem repetição
total_words = []
def lista_sem_repeticao(LISTA,lista):
    resposta = LISTA
    for tweet in lista:
        for palavra in tweet:
            if palavra not in resposta:
                resposta.append(palavra)
    return resposta
lista_sem_repeticao(total_words,groups_tweet[0])
lista_sem_repeticao(total_words,groups_tweet[1])
lista_sem_repeticao(total_words,groups_tweet[2])
N =len(total_words)

In [20]:
N

2970

In [21]:
# Algoritmo de Laplace
def laplace(num_vezes_no_grupo, num_elementos_do_grupo, num_elementos_sem_repeticao):
    probabilidade = (num_vezes_no_grupo + 1 ) / ( num_elementos_do_grupo + num_elementos_sem_repeticao)
    return probabilidade

In [22]:
#Número de vezes que a palavra aparece no grupo
def numero_de_vezes(palavra_analisada,grupo):
    n = 0
    for tweet in grupo:
        for palavra in tweet:
            if palavra ==  palavra_analisada:
                n +=1
    return n

In [23]:
numero_de_vezes("depressão",groups_tweet[1])

122

In [24]:
#Calculando a probabilidade para cada palavra
resultados = []
for group in groups_tweet:
    dictionary= {}
    for tweet in group:
        for word in tweet:
            probabilidade = laplace(numero_de_vezes(word,group),len(group),N)
            dictionary[word] = probabilidade
    resultado = pd.Series(dictionary)
    resultados.append(resultado)

In [25]:
#Número de Palavras em Cada uma dos Grupos
def conta_palavras(grupo):
    n = 0
    for tweet in grupo:
        n += len(tweet)
    return n

In [26]:
N_1 = conta_palavras(groups_tweet[0])
N_2 = conta_palavras(groups_tweet[1])
N_3 = conta_palavras(groups_tweet[2])

___
## Verificando a performance na Planilha Testes

Agora você deve testar o seu classificador com a base de Testes.

In [27]:
teste = planilha_teste["Tweet Text"]

In [28]:
def limpeza_tweet(tweet):
    tweet_limpo = space_cleaning(str(tweet))
    return tweet_limpo

In [29]:
teste_tweet = []
for tweet in teste:
    teste_tweet.append(limpeza_tweet(tweet))

In [30]:
serie_teste = pd.Series(teste_tweet)

In [31]:
len(resultados[0].index)

1898

## Função de Análise

Como os resultados estão muito pequenos, usou-se a função do logarítmo neperiano, ln(x), para os valores ficarem com números mais adequados para o Python.

In [32]:
#Função de Análise de Frase
def analisa_frase(frase):
    condicional_grupo1 = 1
    condicional_grupo2 = 1
    condicional_grupo3 = 1
    lista = [condicional_grupo1, condicional_grupo2,condicional_grupo3]
    
    for palavra in frase:
        if (palavra not in resultados[0].index):
            condicional_grupo1 *= laplace(0,N_1,N)
        else: 
            condicional_grupo1 *= resultados[0][palavra]
            
    for palavra in frase:
        if (palavra not in resultados[1].index):
            condicional_grupo2 *= laplace(0,N_1,N) 
        else:
            condicional_grupo2 *= resultados[1][palavra]
            
    for palavra in frase:
        if (palavra not in resultados[2].index):
            condicional_grupo3 *= laplace(0,N_1,N) 
        else:
            condicional_grupo3 *= resultados[2][palavra]
            
    return [condicional_grupo1,condicional_grupo2,condicional_grupo3]

In [33]:
analisa_frase("Gostei muito de você")

[1.1116983151648638e-60, 4.458025834425154e-61, 1.3518821708300341e-62]

In [34]:
def identifica_valor_maximo(lista_de_resultados):
    if lista_de_resultados[0] == max(lista_de_resultados):
        return 1
    elif lista_de_resultados[1] == max(lista_de_resultados):
        return 2
    else:
        return 3

In [35]:
def resultado(n):
    return identifica_valor_maximo(analisa_frase(limpeza_tweet(teste[n])))
estimativa = []
for k in range(0,len(teste)):
    estimativa.append(resultado(k))

In [36]:
planilha_teste["Relevância_Estimada"] = estimativa

In [37]:
Acertos = 0
for n in planilha_teste.index:
    if planilha_teste["Relevância_Prevista"][n] == planilha_teste["Relevância_Estimada"][n]:
        Acertos += 1

In [38]:
planilha_treinamento["Relevância"].value_counts(True)

1.0    0.492205
2.0    0.285078
3.0    0.222717
Name: Relevância, dtype: float64

In [39]:
planilha_teste["Relevância_Prevista"].value_counts(True)

1    0.536667
3    0.236667
2    0.223333
0    0.003333
Name: Relevância_Prevista, dtype: float64

In [40]:
planilha_teste["Relevância_Estimada"].value_counts(True)

1    0.93
2    0.05
3    0.02
Name: Relevância_Estimada, dtype: float64

In [41]:
Acertos/len(planilha_teste)

0.5466666666666666

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**